The following notebook uses LDA and NMF to get topics from the article content

# Preprocessing

In [ ]:
!pip install nltk
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import GridSearchCV


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
df_read = pd.read_csv("/content/drive/My Drive/Analytics Spring 2020/Work Streams/Core DS/Content Rec/DP Data/dp.csv")

In [ ]:
#Remove articles before 2009 

sample=df_read.sort_values(by=['published_date'])
sample = sample.drop_duplicates()
df = sample.dropna()
df['date'] = pd.to_datetime(df['published_date'])
df1 = df[df.date > datetime.strptime("2009","%Y")]
df1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,id,uuid,title,slug,title_url,content,published_date,date
30680,30825,121752,81c4f5a9-8844-407f-a75f-88d5babf0996,"W. Squash | The coach can play, but can they?",w-squash-the-coach-can-play-but-can-they,2009/01/w-squash-the-coach-can-play-but-can-they,It's a good thing that coaches can't play for ...,2009-01-14 10:00:00,2009-01-14 10:00:00
30679,30824,121753,91b1aa4f-b2f6-4584-b319-56fc94990059,M. Swimming | Hot start in Florida cooled off ...,m-swimming-hot-start-in-florida-cooled-off-by-...,2009/01/m-swimming-hot-start-in-florida-cooled...,To say that the Penn men's swimming team took ...,2009-01-14 10:00:00,2009-01-14 10:00:00
30678,30823,121754,28011c67-1023-4323-a3cc-28dd02d513cb,W. Swimming | Ivy woes continue in the pool,w-swimming-ivy-woes-continue-in-the-pool,2009/01/w-swimming-ivy-woes-continue-in-the-pool,The women's swimming team rang in the New Year...,2009-01-14 10:00:00,2009-01-14 10:00:00
30677,30822,121734,2da73f43-5dec-47e5-bb61-19e8b0b980f2,Sexual assault suspect still at large,sexual_assault_suspect_still_at_large,2009/01/sexual_assault_suspect_still_at_large,Police are still looking for the man who sexua...,2009-01-14 10:00:00,2009-01-14 10:00:00
30676,30821,121756,4c649f87-3ca8-4f53-b6f5-9f14e231a0c3,Sports brief | Gymnastics drops the Ball in op...,sports_brief_gymnastics_drops_the_ball_in_opener,2009/01/sports_brief_gymnastics_drops_the_ball...,"To start its 2009 season, the gymnastics team ...",2009-01-14 10:00:00,2009-01-14 10:00:00
...,...,...,...,...,...,...,...,...,...
85192,204,190843,ce8f0d5c-2f18-41c0-95a7-ea378e5fb068,Philadelphia School District backtracks from i...,philadephia-school-district-online-fall-semest...,2020/07/philadephia-school-district-online-fal...,Following backlash from its initial hybrid reo...,2020-07-31 04:03:10,2020-07-31 04:03:10
85191,203,190851,756bf521-150a-4aaa-be4f-0810dad60c00,"Penn professor wins inaugural $80,000 prize to...",nemo-prize-covid19-test-penn-coronavirus,2020/07/nemo-prize-covid19-test-penn-coronavirus,"César de la Fuente, Penn presidential assistan...",2020-07-31 04:30:46,2020-07-31 04:30:46
85190,202,190854,b85c710d-cd36-405c-bd77-d5c4f86da5e4,Pottruck will reopen under a phased approach f...,pottruck-phased-reopening-fitness-gym-coronavirus,2020/07/pottruck-phased-reopening-fitness-gym-...,"After temporarily closing in mid-March, Pottru...",2020-07-31 05:09:18,2020-07-31 05:09:18
85189,201,190856,c538e24a-d8b5-4ef7-8798-4a18161bc17c,The Trumps continued to spread misinformation ...,donald-trump-jr-2020-election-postpone-misinfo...,2020/07/donald-trump-jr-2020-election-postpone...,"This week, 1968 Wharton graduate and President...",2020-07-31 08:33:29,2020-07-31 08:33:29


Start preprocessing the textual data

In [ ]:
sample_data = df1
import re
sample_data['content_p'] = sample_data['content'].astype(str)
#lower case
sample_data['content_p'] = sample_data['content_p'].map(lambda x: x.lower())
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
pat = r'\b(?:{})\b'.format('|'.join(stop))
#remove punctuation
sample_data['content_p'] = sample_data['content_p'].str.replace(pat, '')
sample_data['content_p'] = sample_data['content_p'].str.replace(r'\s+', ' ')
sample_data['content_p'] = sample_data['content_p'].map(lambda x: re.sub('[,\.!?’‘]', '', x))
#lemmatize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()
remove_list = ['amp', 'http', '19']
def subLemma(word):
  return lemmatizer.lemmatize(word) #lemmatize
  #find_result = list(map(word.find, remove_list))

  #if (sum(find_result) > -1 * len(find_result)):
    #return ''
  #else:
    #return word  
def lemma (input_str):
  input_str=word_tokenize(input_str)
  input_str = list(map(subLemma, input_str))
  return ' '.join(input_str)
  

sample_data['content_p'] = sample_data['content_p'].apply(lambda x: lemma(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#remove numbers
sample_data['content_p'] = sample_data['content_p'].apply(lambda x: re.sub(r'\d+', '', x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
sample_data.to_csv('formatted_content.csv')

In [ ]:
!cp formatted_content.csv "/content/drive/My Drive/CIS520 Project/data set/"

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/CIS520 Project/data set/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/CIS520 Project/data set/test.csv")


# LDA

In [ ]:
#Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.1, min_df=0.001)
# Fit and transform the processed titles
count_data_train = count_vectorizer.fit_transform(df_train['processed_content'])


In [ ]:
texts = df_train['processed_content']
texts.append(df_test['processed_content'])

0       budget cut safety concern west philadelphia sc...
1       week  wharton graduate president donald trump ...
2       césar de la fuente penn presidential assistant...
3       following backlash initial hybrid reopening pl...
4       student concerned enforcement penn campus comp...
                              ...                        
4189    undergraduate fine art program soon come new l...
4190    man charged allegedly raping woman th street j...
4191    construction around campus summer dweller wond...
4192    together undocumented youth network dreamactiv...
4193    total crime saw  percent increase may  may  si...
Name: processed_content, Length: 20966, dtype: object

In [ ]:
count_v = count_vectorizer.fit(texts)

In [ ]:
count_test = count_v.transform(df_test['processed_content'])
count_train = count_v.transform(df_train['processed_content'])

In [ ]:
count_data_test = count_vectorizer.fit_transform(df_train['processed_content'])

In [ ]:

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

# Helper function to print top documents
def print_topics(model, vectorizer, n_top_words, no_top_documents, no_chars, data, documents):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        W = model.transform(data)
       
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
          print(doc_index)
          print (documents[doc_index])

In [ ]:
train_titles= df_train['title'].tolist()
test_titles =df_test['title'].tolist()

In [ ]:
#Train and test LDA for different number of topics and check log likelihood score

from sklearn.decomposition import LatentDirichletAllocation as LDA

lda_10 = LDA(n_components=10, n_jobs=-1, random_state=42)
lda_10.fit(count_train)
lda_10.score(count_test)

-5485675.223648399

In [ ]:
lda_5 = LDA(n_components=5, n_jobs=-1, random_state=42)
lda_5.fit(count_train)
lda_5.score(count_test)

-5477193.592951373

In [ ]:
lda_15 = LDA(n_components=15, n_jobs=-1, random_state=42)
lda_15.fit(count_train)
lda_15.score(count_test)

-5518902.064778693

In [ ]:
lda_7 = LDA(n_components=7, n_jobs=-1, random_state=42)
lda_7.fit(count_train)
lda_7.score(count_test)

-5473151.92258006

In [ ]:
!pip install pyldavis

import pyLDAvis
import pyLDAvis.sklearn

data = pyLDAvis.sklearn.prepare(lda_7, count_train, count_vectorizer)
pyLDAvis.display(data)

Add the LDA topic probabilities to our dataframes and save them for supervised learning

In [ ]:


W_train = lda_7.transform(count_train)
W_test = lda_7.transform(count_test)

In [ ]:
df_test['prob_LDAtopic_1']=W_test[:,:1]
df_test['prob_LDAtopic_2']=W_test[:,1:2]
df_test['prob_LDAtopic_3']=W_test[:,2:3]
df_test['prob_LDAtopic_4']=W_test[:,3:4]
df_test['prob_LDAtopic_5']=W_test[:,4:5]
df_test['prob_LDAtopic_6']=W_test[:,5:6]
df_test['prob_LDAtopic_7']=W_test[:,6:7]

df_train['prob_LDAtopic_1']=W_train[:,:1]
df_train['prob_LDAtopic_2']=W_train[:,1:2]
df_train['prob_LDAtopic_3']=W_train[:,2:3]
df_train['prob_LDAtopic_4']=W_train[:,3:4]
df_train['prob_LDAtopic_5']=W_train[:,4:5]
df_train['prob_LDAtopic_6']=W_train[:,5:6]
df_train['prob_LDAtopic_7']=W_train[:,6:7]


df_train

,id,title,title_url,content,published_date,article,pageviews,avgTimeOnPage,days,processed_content,year_month,percentile,prob_LDAtopic_1,prob_LDAtopic_2,prob_LDAtopic_3,prob_LDAtopic_4,prob_LDAtopic_5,prob_LDAtopic_6,prob_LDAtopic_7
0,190855,Morgan Gilfond | Philadelphia schools still ha...,philadelphia-schools-still-havent-removed-asbe...,Budget cuts and safety concerns in the West Ph...,2020-07-31 13:11:27,/article/2020/07/philadelphia-schools-still-ha...,262.0,209.444444,7,budget cut safety concern west philadelphia sc...,2020-07,0.145570,0.000574,0.000574,0.184160,0.290273,0.253202,0.270643,0.000574
1,190856,The Trumps continued to spread misinformation ...,donald-trump-jr-2020-election-postpone-misinfo...,"This week, 1968 Wharton graduate and President...",2020-07-31 08:33:29,/article/2020/07/donald-trump-jr-2020-election...,289.0,144.261745,7,week wharton graduate president donald trump ...,2020-07,0.177215,0.000463,0.000463,0.127464,0.030435,0.766332,0.074379,0.000464
2,190851,"Penn professor wins inaugural $80,000 prize to...",nemo-prize-covid19-test-penn-coronavirus,"César de la Fuente, Penn presidential assistan...",2020-07-31 04:30:46,/article/2020/07/nemo-prize-covid19-test-penn-...,540.0,165.496000,7,césar de la fuente penn presidential assistant...,2020-07,0.360759,0.000980,0.000983,0.000983,0.000983,0.005451,0.989637,0.000984
3,190843,Philadelphia School District backtracks from i...,philadephia-school-district-online-fall-semest...,Following backlash from its initial hybrid reo...,2020-07-31 04:03:10,/article/2020/07/philadephia-school-district-o...,224.0,202.011364,7,following backlash initial hybrid reopening pl...,2020-07,0.088608,0.001015,0.001015,0.128298,0.331011,0.016777,0.520868,0.001016
4,190849,Students question enforcement of Campus Compac...,penn-student-campus-compact-fall-return-to-campus,Students are concerned over the enforcement of...,2020-07-31 03:48:02,/article/2020/07/penn-student-campus-compact-f...,1599.0,217.869565,7,student concerned enforcement penn campus comp...,2020-07,0.696203,0.000475,0.000474,0.216641,0.756817,0.000475,0.000475,0.024643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16767,131968,Students petition Medical School to reinstate ...,students_petition_medical_school_to_reinstate_...,"Amid protests in a student-led petition, the P...",2012-06-27 13:53:23,/article/2012/06/students_petition_medical_sch...,189.0,210.089109,7,amid protest student-led petition perelman sch...,2012-06,0.900000,0.000559,0.000560,0.000561,0.750526,0.000560,0.246674,0.000560
16768,131967,Man fatally shot on 40th & Market and 7/11 is ...,fatal_shooting_at_40th_and_market,A man was fatally shot on 40th and Market stre...,2012-06-26 15:12:31,/article/2012/06/fatal_shooting_at_40th_and_ma...,17.0,63.666667,7,man fatally shot th market street june june ...,2012-06,0.500000,0.001726,0.001734,0.989645,0.001724,0.001724,0.001724,0.001724
16769,131933,PennMOVES sale turns trash into treasure,pennmoves_sale_turns_trash_into_treasure,"With items as cheap as a few dollars, PennMOVE...",2012-06-07 02:11:44,/article/2012/06/pennmoves_sale_turns_trash_in...,1.0,0.000000,7,item cheap dollar pennmoves every philadelphia...,2012-06,0.150000,0.001314,0.001317,0.418440,0.001317,0.001314,0.260541,0.315755
16770,131927,First Lady Michelle Obama rallies in Philadelphia,first_lady_rallies_in_philadelphia,First Lady Michelle Obama addressed a crowd of...,2012-06-07 01:49:59,/article/2012/06/first_lady_rallies_in_philade...,11.0,25.909091,7,first lady michelle obama addressed crowd appr...,2012-06,0.400000,0.000869,0.000870,0.000870,0.000869,0.945321,0.000871,0.050330


In [ ]:
df_test.to_csv("test.csv")

In [ ]:
!cp test.csv "/content/drive/MyDrive/CIS520 Project/"

Seeing the LDA on topics 20 and how its topics look like

In [ ]:
lda_20 = LDA(n_components=20, n_jobs=-1, random_state=42)
lda_20.fit(count_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
!pip install pyldavis

import pyLDAvis
import pyLDAvis.sklearn

data = pyLDAvis.sklearn.prepare(lda_20, count_train, count_vectorizer)
pyLDAvis.display(data)

# NMF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#TFIDF

vect = TfidfVectorizer(stop_words='english',max_df=0.7, min_df=20)
tfidf = vect.fit_transform(df_train['processed_content'])
tfidf_test = vect.transform(df_test['processed_content'])

Now apply NMF on different number of topics and see their keywords and top documents

In [ ]:
num_topics = 5
number_words = 20
number_documents = 10
from sklearn.decomposition import NMF
model = NMF(n_components=num_topics, init='random', random_state=0)
W = model.fit_transform(tfidf)
H = model.components_

In [ ]:

print("Topics: ")
print(num_topics)
print_topics(model, vect, number_words, number_documents,150,tfidf, train_titles)
print("Test")
print_topics(model, vect, number_words, number_documents,150,tfidf_test, test_titles)

Topics: 
15

Topic #0:
ivy team season league game year princeton tournament player conference harvard coach title quaker win play yale championship ncaa basketball
2845
A Decade in Ivy Sports: Which rivalries are the most competitive? 
1284
Harvard is the favorite, but what can we expect from the other Ivy men's teams?
7987
Tydings | Taking a look at the planned Ivy League basketball tournament
3550
Which Ivy League teams have had the most success over the past decade?
8603
Women's Lacrosse Issue | Expectations still the same despite end of reign atop the Ivy League
581
Ivy League men's basketball power rankings: Week 7
16539
Ghosh | Penn women's soccer has no room for error
10969
Tydings | Time for an Ivy League basketball tournament
8814
Steele | 14 games are good enough
3500
Which Ivy League team has had the best individual season?

Topic #1:
game goal quaker yard field touchdown red defense football play blue ball minute offense half second quarterback season score win
7164
Penn f

In [ ]:
num_topics = 7
number_words = 20
number_documents = 10
from sklearn.decomposition import NMF
model = NMF(n_components=num_topics, init='random', random_state=0)
W = model.fit_transform(tfidf)
H = model.components_

In [ ]:

print("Topics: ")
print(num_topics)
print_topics(model, vect, number_words, number_documents,150,tfidf, train_titles)
print("Test")
print_topics(model, vect, number_words, number_documents,150,tfidf_test, test_titles)

Topics: 
7

Topic #0:
game quaker point half minute red play blue ivy season goal team win basketball shot guard second lead ball donahue
4286
Penn men's basketball clinches Ivy tournament berth with wins over Columbia, Cornell
798
Penn men's basketball defeats Columbia, 76-67, in tight Saturday afternoon contest
11016
Penn basketball loses 15-point lead, drops Ivy opener to rival Princeton
4613
Penn men's basketball survives late game scare to defeat Cornell, 69-61
729
Penn outlasts Ivy League-leading Yale, 69-61, with strong second half play
2534
Three things we learned from Penn women's basketball's 54-48 win over Yale
2867
Penn women's basketball splits pair of close contests at Navy Classic
678
Padilla powers Penn women's basketball to seventh Ivy win, 70-48 over Harvard
2349
Behind runs of 14 and 13 points, Penn men's basketball beats Dartmouth, 65-51
2732
Penn men's basketball falls to Princeton again, 62-53, for fourth straight loss

Topic #1:
police street theft suspect unaffi

In [ ]:
num_topics = 10
number_words = 20
number_documents = 10
from sklearn.decomposition import NMF
model = NMF(n_components=num_topics, init='random', random_state=0)
W = model.fit_transform(tfidf)
H = model.components_

In [ ]:

print("Topics: ")
print(num_topics)
print_topics(model, vect, number_words, number_documents,150,tfidf, train_titles)
print("Test")
print_topics(model, vect, number_words, number_documents,150,tfidf_test, test_titles)

Topics: 
10

Topic #0:
yard football game touchdown quarterback bagnoli priore receiver watson quaker torgersen season offense play quarter defense defensive field drive running
7764
Retelling the narrative arc of Penn football's 2015 championship
5162
Penn football falls to Columbia in overtime, 34-31
9075
Penn football clinches Ivy League title with win over Cornell
7236
Penn football routs Yale, 42-7
9155
Penn football preps for clash with Harvard with title on line
16383
Penn football holds off Big Green in Ivy opener
1304
Penn football looks to pick up first Ivy League win against struggling Brown
16188
Penn football drops first Ivy contest at Yale
13798
Penn football defeats Columbia in ugly affair, 21-7
5098
Penn sprint football wallops Post in final tune-up before pivotal Navy clash

Topic #1:
game quaker goal ivy season win team red blue play minute league second weekend shot inning half field soccer defense
1911
Top-seeded Penn men's lacrosse edges past Brown in Ivy League To

In [ ]:
num_topics = 15
number_words = 20
number_documents = 10
from sklearn.decomposition import NMF
model = NMF(n_components=num_topics, init='random', random_state=0)
W = model.fit_transform(tfidf)
H = model.components_

In [ ]:

print("Topics: ")
print(num_topics)
print_topics(model, vect, number_words, number_documents,150,tfidf, train_titles)
print("Test")
print_topics(model, vect, number_words, number_documents,150,tfidf_test, test_titles)

Topics: 
15

Topic #0:
ivy team season league game year princeton tournament player conference harvard coach title quaker win play yale championship ncaa basketball
2845
A Decade in Ivy Sports: Which rivalries are the most competitive? 
1284
Harvard is the favorite, but what can we expect from the other Ivy men's teams?
7987
Tydings | Taking a look at the planned Ivy League basketball tournament
3550
Which Ivy League teams have had the most success over the past decade?
8603
Women's Lacrosse Issue | Expectations still the same despite end of reign atop the Ivy League
581
Ivy League men's basketball power rankings: Week 7
16539
Ghosh | Penn women's soccer has no room for error
10969
Tydings | Time for an Ivy League basketball tournament
8814
Steele | 14 games are good enough
3500
Which Ivy League team has had the best individual season?

Topic #1:
game goal quaker yard field touchdown red defense football play blue ball minute offense half second quarterback season score win
7164
Penn f

In [ ]:
#Export NMF results to be used in Supervised Learning
df_test_upsampled = pd.read_csv('/content/drive/MyDrive/CIS520 Project/data set/test_df_upsampled.csv', index_col=False)

In [ ]:
tfidf_test_upsampled = vect.transform(df_test_upsampled['processed_content'])

In [ ]:
W_train = model.transform(tfidf)
W_test = model.transform(tfidf_test_upsampled)


df_test_upsampled['prob_NMFtopic_1']=W_test[:,:1]
df_test_upsampled['prob_NMFtopic_2']=W_test[:,1:2]
df_test_upsampled['prob_NMFtopic_3']=W_test[:,2:3]
df_test_upsampled['prob_NMFtopic_4']=W_test[:,3:4]
df_test_upsampled['prob_NMFtopic_5']=W_test[:,4:5]
df_test_upsampled['prob_NMFtopic_6']=W_test[:,5:6]
df_test_upsampled['prob_NMFtopic_7']=W_test[:,6:7]
df_test_upsampled['prob_NMFtopic_8']=W_test[:,7:8]
df_test_upsampled['prob_NMFtopic_9']=W_test[:,8:9]
df_test_upsampled['prob_NMFtopic_10']=W_test[:,9:10]
df_test_upsampled['prob_NMFtopic_11']=W_test[:,10:11]
df_test_upsampled['prob_NMFtopic_12']=W_test[:,11:12]
df_test_upsampled['prob_NMFtopic_13']=W_test[:,12:13]
df_test_upsampled['prob_NMFtopic_14']=W_test[:,13:14]
df_test_upsampled['prob_NMFtopic_15']=W_test[:,14:15]


df_train['prob_NMFtopic_1']=W_train[:,:1]
df_train['prob_NMFtopic_2']=W_train[:,1:2]
df_train['prob_NMFtopic_3']=W_train[:,2:3]
df_train['prob_NMFtopic_4']=W_train[:,3:4]
df_train['prob_NMFtopic_5']=W_train[:,4:5]
df_train['prob_NMFtopic_6']=W_train[:,5:6]
df_train['prob_NMFtopic_7']=W_train[:,6:7]
df_train['prob_NMFtopic_8']=W_train[:,7:8]
df_train['prob_NMFtopic_9']=W_train[:,8:9]
df_train['prob_NMFtopic_10']=W_train[:,9:10]
df_train['prob_NMFtopic_11']=W_train[:,10:11]
df_train['prob_NMFtopic_12']=W_train[:,11:12]
df_train['prob_NMFtopic_13']=W_train[:,12:13]
df_train['prob_NMFtopic_14']=W_train[:,13:14]
df_train['prob_NMFtopic_15']=W_train[:,14:15]



df_train

,Unnamed: 0,id,title,title_url,content,published_date,article,pageviews,avgTimeOnPage,days,processed_content,year_month,percentile,prob_LDAtopic_1,prob_LDAtopic_2,prob_LDAtopic_3,prob_LDAtopic_4,prob_LDAtopic_5,prob_LDAtopic_6,prob_LDAtopic_7,prob_NMFtopic_1,prob_NMFtopic_2,prob_NMFtopic_3,prob_NMFtopic_4,prob_NMFtopic_5,prob_NMFtopic_6,prob_NMFtopic_7,prob_NMFtopic_8,prob_NMFtopic_9,prob_NMFtopic_10,prob_NMFtopic_11,prob_NMFtopic_12,prob_NMFtopic_13,prob_NMFtopic_14,prob_NMFtopic_15
0,0,190855,Morgan Gilfond | Philadelphia schools still ha...,philadelphia-schools-still-havent-removed-asbe...,Budget cuts and safety concerns in the West Ph...,2020-07-31 13:11:27,/article/2020/07/philadelphia-schools-still-ha...,262.0,209.444444,7,budget cut safety concern west philadelphia sc...,2020-07,0.145570,0.000574,0.000574,0.184160,0.290273,0.253202,0.270643,0.000574,0.000000,0.000000,0.003155,0.040372,0.002504,0.026954,0.000000,0.000023,0.000000,0.005197,0.006190,0.000000,0.016433,0.000000e+00,0.000000
1,1,190856,The Trumps continued to spread misinformation ...,donald-trump-jr-2020-election-postpone-misinfo...,"This week, 1968 Wharton graduate and President...",2020-07-31 08:33:29,/article/2020/07/donald-trump-jr-2020-election...,289.0,144.261745,7,week wharton graduate president donald trump ...,2020-07,0.177215,0.000463,0.000463,0.127464,0.030435,0.766332,0.074379,0.000464,0.001124,0.000000,0.000000,0.002222,0.106454,0.003477,0.000000,0.000000,0.000000,0.004600,0.000000,0.000208,0.001056,0.000000e+00,0.000000
2,2,190851,"Penn professor wins inaugural $80,000 prize to...",nemo-prize-covid19-test-penn-coronavirus,"César de la Fuente, Penn presidential assistan...",2020-07-31 04:30:46,/article/2020/07/nemo-prize-covid19-test-penn-...,540.0,165.496000,7,césar de la fuente penn presidential assistant...,2020-07,0.360759,0.000980,0.000983,0.000983,0.000983,0.005451,0.989637,0.000984,0.000000,0.000000,0.005756,0.018573,0.000000,0.000718,0.000429,0.001827,0.000000,0.000000,0.000000,0.000799,0.010619,0.000000e+00,0.000000
3,3,190843,Philadelphia School District backtracks from i...,philadephia-school-district-online-fall-semest...,Following backlash from its initial hybrid reo...,2020-07-31 04:03:10,/article/2020/07/philadephia-school-district-o...,224.0,202.011364,7,following backlash initial hybrid reopening pl...,2020-07,0.088608,0.001015,0.001015,0.128298,0.331011,0.016777,0.520868,0.001016,0.000000,0.000000,0.000694,0.024242,0.000000,0.039038,0.000000,0.000000,0.000000,0.000734,0.000000,0.000000,0.023727,0.000000e+00,0.000000
4,4,190849,Students question enforcement of Campus Compac...,penn-student-campus-compact-fall-return-to-campus,Students are concerned over the enforcement of...,2020-07-31 03:48:02,/article/2020/07/penn-student-campus-compact-f...,1599.0,217.869565,7,student concerned enforcement penn campus comp...,2020-07,0.696203,0.000475,0.000474,0.216641,0.756817,0.000475,0.000475,0.024643,0.000000,0.000000,0.003929,0.000000,0.002588,0.038264,0.000000,0.000000,0.028415,0.001853,0.001062,0.000000,0.026561,0.000000e+00,0.001112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16767,16767,131968,Students petition Medical School to reinstate ...,students_petition_medical_school_to_reinstate_...,"Amid protests in a student-led petition, the P...",2012-06-27 13:53:23,/article/2012/06/students_petition_medical_sch...,189.0,210.089109,7,amid protest student-led petition perelman sch...,2012-06,0.900000,0.000559,0.000560,0.000561,0.750526,0.000560,0.246674,0.000560,0.000000,0.000000,0.000000,0.020361,0.001553,0.044281,0.000000,0.000000,0.012796,0.000000,0.000000,0.000000,0.014336,0.000000e+00,0.000000
16768,16768,131967,Man fatally shot on 40th & Market and 7/11 is ...,fatal_shooting_at_40th_and_market,A man was fatally shot on 40th and Market stre...,2012-06-26 15:12:31,/article/2012/06/fatal_shooting_at_40th_and_ma...,17.0,63.666667,7,man fa

In [ ]:
df_test_upsampled.to_csv("NMF_test.csv")
!cp NMF_test.csv "/content/drive/MyDrive/CIS520 Project/data set/"
df_train.to_csv("NMF_train.csv")
!cp NMF_train.csv "/content/drive/MyDrive/CIS520 Project/data set/"